In [1]:
import numpy as np
import pandas as pd 
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
network=pd.read_csv('../data/raw_files/Network_PUF.csv',low_memory=False, encoding ='latin1')
#businessyear and issuerID only numerical vals
network.head()

,BusinessYear,StateCode,IssuerId,SourceName,ImportDate,NetworkName,NetworkId,NetworkURL,MarketCoverage,DentalOnlyPlan
0,2020,AK,21989,HIOS,2019-08-09 03:20:19,Delta Dental Premier,AKN001,https://www.modahealth.com/ProviderSearch?prod...,Individual,Yes
1,2020,AK,21989,HIOS,2019-08-09 03:20:19,Delta Dental Premier,AKN001,https://www.modahealth.com/ProviderSearch?prod...,SHOP (Small Group),Yes
2,2020,AK,21989,HIOS,2019-08-09 03:20:19,Delta Dental PPO,AKN002,https://www.modahealth.com/ProviderSearch?prod...,Individual,Yes
3,2020,AK,21989,HIOS,2019-08-09 03:20:19,Delta Dental PPO,AKN002,https://www.modahealth.com/ProviderSearch?prod...,SHOP (Small Group),Yes
4,2020,AK,38344,HIOS,2019-08-22 03:20:26,Legacy and Dental Select,AKN001,https://premera.sapphirecareselect.com/?ci=pre...,Individual,No


In [3]:
network.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 648 entries, 0 to 647
Data columns (total 10 columns):
BusinessYear      648 non-null int64
StateCode         648 non-null object
IssuerId          648 non-null int64
SourceName        648 non-null object
ImportDate        648 non-null object
NetworkName       648 non-null object
NetworkId         648 non-null object
NetworkURL        648 non-null object
MarketCoverage    648 non-null object
DentalOnlyPlan    648 non-null object
dtypes: int64(2), object(8)
memory usage: 50.7+ KB


In [4]:
#network['DentalOnlyPlan'].unique()


#this gets rid of the Dental Only Plans leaving medical insurance plans then drops the column as superfilous
network=network[network.DentalOnlyPlan != 'Yes']
network=network.drop('DentalOnlyPlan', axis=1)

network=network.drop(['BusinessYear','SourceName', 'ImportDate','NetworkURL'], axis = 1)
network.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 291 entries, 4 to 647
Data columns (total 5 columns):
StateCode         291 non-null object
IssuerId          291 non-null int64
NetworkName       291 non-null object
NetworkId         291 non-null object
MarketCoverage    291 non-null object
dtypes: int64(1), object(4)
memory usage: 13.6+ KB


In [5]:
network_FL=network[network.StateCode == 'FL'].copy()
network_FL.drop(['StateCode','MarketCoverage'],axis=1, inplace=True)
network_FL.columns
network_FL=network_FL[['IssuerId','NetworkId','NetworkName']]


In [6]:
fl_uniques={} 
for col in network_FL.columns: 
    fl_uniques[col]=network_FL[col].unique()
fl_uniques

{'IssuerId': array([12379, 16842, 21663, 30252, 36194, 40572, 48121, 54172, 56503]),
 'NetworkId': array(['FLN001', 'FLN002', 'FLN003', 'FLN004'], dtype=object),
 'NetworkName': array(['Bright Health Network Orlando',
        'Bright Health Network Jacksonville/Pensacola',
        'Bright Health Network Tampa', 'Bright Health Network Palm Beach',
        'NetworkBlue', 'BlueSelect',
        'Ambetter from Sunshine Health Network', 'BlueCare', 'myBlue',
        'Health First Individual', 'AdventHealth Individual',
        'Florida EPO', 'Florida Connect Network', 'Molina Marketplace',
        'FHCP HMO Network'], dtype=object)}

In [7]:
for key,value in fl_uniques.items():
    print(key, len(value))

IssuerId 9
NetworkId 4
NetworkName 15


In [10]:
network_FL


,IssuerId,NetworkId,NetworkName
42,12379,FLN001,Bright Health Network Orlando
43,12379,FLN002,Bright Health Network Jacksonville/Pensacola
44,12379,FLN003,Bright Health Network Tampa
45,12379,FLN004,Bright Health Network Palm Beach
49,16842,FLN001,NetworkBlue
50,16842,FLN002,BlueSelect
52,21663,FLN001,Ambetter from Sunshine Health Network
56,30252,FLN001,BlueCare
57,30252,FLN002,myBlue
58,36194,FLN001,Health First Individual


In [9]:
#network_FL.reset_index()
network_FL.to_csv('../data/clean_files/network_FL.csv')

In [18]:
zip=33101
area_import = pd.read_csv('../data/clean_files/area_FL.csv')
area=area_import.copy()
area.drop(['Unnamed: 0','County','County_Name','ServiceAreaId'], axis=1, inplace=True)
area.dropna(how='any', inplace=True)
area['IssuerId']=area['IssuerId'].astype(int)
area=area[area['zipcode'] ==zip]
area=area.drop_duplicates(['IssuerId'])
matrix=pd.read_pickle('../models/florida_jaccard.pkl')
area_plans=area.merge(matrix, how='left', on='IssuerId')
area_plans.drop('zipcode', axis=1, inplace=True)


In [21]:
area

,zipcode,IssuerId
4182,33101,16842
4183,33101,21663
4184,33101,30252
4185,33101,40572
4186,33101,54172
